In [1]:
import pandas as pd 
import requests 
import json 
from google.cloud import bigquery 
from urllib.parse import quote, unquote
import os
import configparser

In [2]:
configfile = configparser.ConfigParser()
configfile.read("config.cfg")
print(configfile.sections())

# set SSL cert vals, not necessary unless there's SSL cert error
os.environ["REQUESTS_CA_BUNDLE"] = configfile['SSL']['REQUESTS_CA_BUNDLE']
os.environ["SSL_CERT_FILE"] = configfile['SSL']['SSL_CERT_FILE']

# google IAM credentials

['SSL', 'GOOGLE']


In [3]:
## get data from open data 

base_url = "https://data.cityofnewyork.us/resource/h9gi-nx95.json?"
api_query = """SELECT 
                    collision_id,
                    crash_date,
                    crash_time,
                    borough,
                    zip_code, 
                    latitude,
                    longitude,
                    location,
                    on_street_name,
                    cross_street_name,
                    off_street_name, 
                    number_of_persons_injured,
                    number_of_persons_killed,
                    number_of_pedestrians_injured,
                    number_of_pedestrians_killed,
                    number_of_cyclist_injured,
                    number_of_cyclist_killed,
                    number_of_motorist_injured,
                    number_of_motorist_killed,
                    contributing_factor_vehicle_1,
                    contributing_factor_vehicle_2,
                    vehicle_type_code1,
                    vehicle_type_code2
                LIMIT 10000
                
            """

In [4]:
url = base_url + "$query=" + api_query
resp = requests.get(url)

In [5]:
df_opendata = pd.DataFrame(resp.json())

In [6]:
df_opendata[['crash_date', 'crash_time']]

,crash_date,crash_time
0,2021-09-11T00:00:00.000,2:39
1,2022-03-26T00:00:00.000,11:45
2,2023-11-01T00:00:00.000,1:29
3,2022-06-29T00:00:00.000,6:55
4,2022-09-21T00:00:00.000,13:21
...,...,...
9995,2021-09-18T00:00:00.000,10:50
9996,2021-09-18T00:00:00.000,1:00
9997,2021-09-17T00:00:00.000,9:10
9998,2021-09-18T00:00:00.000,4:48
